### Fasion MNIST (CNN)

In [1]:
# import libraries
import torch
from torch.optim import Adam
from torch.nn import Linear, MSELoss, Sequential, Sigmoid, Module
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

import seaborn as sb
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

from tensorflow.keras.datasets.fashion_mnist import load_data

In [2]:
# load data
(x_train, y_train),(x_test,y_test) = load_data()

In [3]:
# list of label name
text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']

In [4]:
# flatten
sample_img = torch.zeros(1, 1, 28, 28)
conv1 = nn.Conv2d(in_channels=1, out_channels=30, kernel_size=5, stride=1)
linear = nn.Linear(17280, 2) # z = matmul(x, w) + b / w: 17280 * 2 / b: 2
x = conv1(sample_img)
print(x.shape) # (28 - 5) / 1 + 1
x = torch.flatten(x, 1) # x.view(1, 30 * 24 * 24)
print('flatten', x.shape)
linear(x)

torch.Size([1, 30, 24, 24])
flatten torch.Size([1, 17280])


tensor([[ 0.0709, -0.0916]], grad_fn=<AddmmBackward>)

In [5]:
# CNN Class
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=30, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=30, out_channels=60, kernel_size=5, stride=1)        
        self.max1 = nn.MaxPool2d(kernel_size=2)
        example_input = torch.zeros(1, 1, 28, 28)  # MNIST image size
        flattened_size = self._get_flattened_size(example_input)
        print('flattened_size=', flattened_size)
        self.fc1 = nn.Linear(flattened_size, 50)
        self.fc2 = nn.Linear(50, 10)
        self.smax = nn.Softmax(dim=1)

    def _get_flattened_size(self, x):
        with torch.no_grad(): # 학습과 관계없는 텐서 크기 계산이므로 no_grad 사용
            x = F.relu(self.conv1(x))
            x = F.relu(self.conv2(x))
            x =  self.max1(x)
            x = torch.flatten(x, 1)
            return torch.flatten(x, 1).shape[1]  
            
    def forward(self, x):
        x = F.relu(self.conv1(x)) 
        x = F.relu(self.conv2(x))
        x =  self.max1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.smax(x)
        return x

In [6]:
test = torch.IntTensor( [[1,2],[3,4]])
t = torch.IntTensor( [0,1])
pred = test.max(dim=1)[1] # 행에서 가장 큰 값의 인덱스
print(pred)
(pred == t).sum()

tensor([1, 1])


tensor(1)

In [7]:
x1 = x_train.reshape(-1,1,28,28)
x = torch.FloatTensor(x1/255)
y = torch.LongTensor(y_train)

dataset = TensorDataset(x, y)  # TensorDataset으로 데이터셋 생성
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)  # DataLoader로 배치 구성

C:\Users\wsadm\AppData\Local\Temp\ipykernel_15088\1167537053.py:3: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  y = torch.LongTensor(y_train)


In [ ]:
# training
model = CNN().to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = Adam( model.parameters(), lr=0.01)

for step in range(5):
    model.train()
    for batch_idx, (batch_x, batch_y) in enumerate(dataloader):
        optimizer.zero_grad()
        hx = model(batch_x)  # 배치 데이터로 예측
        cost = loss_fn(hx, batch_y)  # 손실 계산
        cost.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트
        # print('batch',batch_idx)

    model.eval()
    correct = 0
    with torch.no_grad() :
        # 데이터의 수 만큼 반복한다.
        for i, (data, target) in enumerate(dataloader) :
            # data = data
            # target = target
            output = model(data)

            # pred = output.max(1, keepdim=True)[1]
            # a = pred.eq(target.view_as(pred)).sum().item()
            pred = output.max(dim=1)[1]
            correct += (pred==target).sum().item()
            
    # 맞춘 개수를 전체 개수로 나눠 맞춘 확률을 구한다.
    test_accuracy = 100.0 * correct / len(dataloader.dataset) 
    print('정확도', test_accuracy)

flattened_size= 6000
정확도 60.84
정확도 65.14833333333333
정확도 56.19166666666667
정확도 63.63166666666667
정확도 59.945
